In [6]:
import json, os, re
import lxml.etree as ET
#!pip install dateparser                                 # Attention ! installer dateparser pour la reconnaissance des dates


# Chemin du dossier contenant les fichiers à parcourir
dossier_json = "C:/Users/brune/Desktop/stage_AGODA/json_data"
dossier_xml = "C:/Users/brune/Desktop/stage_AGODA/xml_data"

# Itérer sur tous les fichiers dans le dossier
for filename in os.listdir(dossier_json): 
    # Vérifier si le fichier est un fichier JSON
    if filename.endswith(".json"):
        # Ouvrir le fichier JSON
        with open(os.path.join(dossier_json, filename), "r", encoding="utf-8") as json_file:
            # Lire le contenu du fichier JSON
            data = json.load(json_file)

            #Création de l'élément racine et le squelette du xml
            
 # Création l'élément racine avec le namespace xmlns
            # root = ET.Element("{http://www.tei-c.org/ns/1.0}TEI", nsmap={None: "http://www.tei-c.org/ns/1.0"})
            root = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
            
            # Définir l'attribut xml:lang
            root.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"
            
            # Ajout de l'attribut xml:id avec la valeur du nom du fichier sans l'extension
            xml_id = filename.split(".")[0]
            root.attrib["{http://www.w3.org/XML/1998/namespace}id"] = xml_id
            
            teiHeader = ET.SubElement(root, "teiHeader")
            text_tei = ET.SubElement(root, "text")
            body = ET.SubElement(text_tei, "body")
            

            # ajout de la variable page_number qui récupère si possible le numéro de page dans le nom du fichier
            page_number = filename.split("_p0")[1].split(".")[0]
   


 # Ajout des éléments de la TeiHeader ________________________________________________________________________________
        
        # fileDesc______________________________________________________________________
        
            fileDesc = ET.SubElement(teiHeader , "fileDesc")
            
            # Titre______________________________________________________________________
        
            titleStmt = ET.SubElement(fileDesc , "titleStmt")
            title_fr = ET.SubElement(titleStmt, "title", type ="main")
            title_fr.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"
            title_en = ET.SubElement(titleStmt, "title", type ="main")
            title_en.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "en"
            
            title_sub_fr = ET.SubElement(titleStmt, "title", type ="sub")
            title_sub_fr.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"
            title_sub_en = ET.SubElement(titleStmt, "title", type ="sub")
            title_sub_en.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "en"         
            
            # meeting____________________________________________________
            
                # voir partie Meeting plus bas

            # respStmt________________________________________________________
            
            personnes = {"Brunel TCHEKELI": "id-hal", "Marie PUREN": "id-hal", "Pierre VERNUS": "id-hal"}
            
            # Boucle pour créer respStmt pour chaque personne
            for personne, identifiant in personnes.items():
                respStmt = ET.SubElement(titleStmt , "respStmt")
                persName = ET.SubElement(respStmt , "persName")
                forename = ET.SubElement(persName , "forename")
                surname = ET.SubElement(persName , "surname")

                # Définir l'attribut xml:id pour l'élément ptr
                #ptr = ET.SubElement(persName, "ptr")
                #ptr.attrib["type"] = identifiant

                 # Définir le texte pour forename et surname
                forename.text = personne.split()[0]  # Utiliser le prénom de la personne
                surname.text = personne.split()[1]  # Utiliser le nom de famille de la personne

                # Vérifier si la personne est "Marie PUREN"
                if personne == "Brunel TCHEKELI":
                    # Ajouter l'élément ptr avec les attributs appropriés
                    resp_fr = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})                        
                    resp_fr.text = "Transformation du JSON en XML-TEI et ajout automatique des balises TEI par des scripts Python"
                    resp_en = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
                    resp_en.text = "Transformation from JSON to XML-TEI and automatic addition of TEI tags by Python scripts"
                
                # Vérifier si la personne est "Marie PUREN"
                if personne == "Marie PUREN":
                    # Ajouter l'élément ptr avec les attributs appropriés
                    ptr = ET.SubElement(persName, "ptr", type= identifiant, target=personne)
                    ptr = ET.SubElement(persName, "ptr", type="orcid", target="0000-0001-5452-3913")
                    resp_fr = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})                        
                    resp_fr.text = "TEI Header"
                    resp_en = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
                    resp_en.text = "TEI Header"

                else:
                    # Ajouter l'élément ptr avec les attributs standard
                    ptr = ET.SubElement(persName, "ptr", type=identifiant, target=personne)

           

            # Création de l'élément funder
            funder = ET.SubElement(titleStmt, "funder")
            # Création des éléments orgName avec les attributs xml:lang correspondants
            orgName_fr = ET.SubElement(funder, "orgName", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})
            orgName_fr.text = "Bibliothèque nationale de France"
            orgName_en = ET.SubElement(funder, "orgName", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
            orgName_en.text = "National Library of France"
            
            # Ajout de extent avec les informations sur le nombre de page, le nombre de mots etc (plus tard)                      
                            
            extent= ET.SubElement(fileDesc, "extent")
            measure_pages_fr = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "fr"})
            measure_pages_fr.text = "pages"
            measure_pages_en = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "en"})
            measure_pages_en.text = "pages"
            
            measure_utterances_sl = ET.SubElement(extent, "measure", {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "sl"})
            measure_utterances_sl.text = "énoncés"
            measure_utterances_en = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "en"})
            measure_utterances_en.text = "utterances"
            
            measure_words_sl = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "sl"})
            measure_words_sl.text = "mots"
            measure_words_en = ET.SubElement(extent, "measure", {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "en"})
            measure_words_en.text = "words"
            
            
            publicationStmt = ET.SubElement(fileDesc , "publicationStmt")
            publisher = ET.SubElement(publicationStmt, "publisher")
            publisher.text = "AGODA"
            authority = ET.SubElement(publicationStmt, "authority")
            authority.text = "Bnf Datalab"
            availability = ET.SubElement(publicationStmt, "availability", status="restricted", n="cc-by")
            licence = ET.SubElement(availability, "licence", target="https://creativecommons.org/licenses/by/4.0/")
            
            # ajout de la date _________________________________
            import datetime
            now = datetime.datetime.now()
            date = ET.SubElement(publicationStmt, "date", {"when": now.strftime("%Y-%m-%d")})
            # date générée automatiquement en utilisant la méthode now() de la classe datetime.datetime et
            # le format est défini avec strftime() en utilisant le modèle "AAAA-MM-JJ"
            
             # ajout de la dsourceDesc _________________________________
            sourceDesc = ET.SubElement(fileDesc , "sourceDesc")
            biblFull = ET.SubElement(sourceDesc, "biblFull")
            titleStmt_sDc = ET.SubElement(biblFull, "titleStmt")
            title_sDc = ET.SubElement(titleStmt_sDc, "title")
            title_sDc.text = "A définir"
            
            publicationStmt_sDc = ET.SubElement(biblFull , "publicationStmt")
            publisher_sDc_fr = ET.SubElement(publicationStmt_sDc, "publisher", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})
            publisher_sDc_fr.text = " "
            
            publisher_sDc_en = ET.SubElement(publicationStmt_sDc, "publisher", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
            publisher_sDc_en.text = " "
            
            pubPlace_sDc = ET.SubElement(publicationStmt_sDc, "pubPlace")
            location_sDc = ET.SubElement(pubPlace_sDc, "location")
            country_sDc = ET.SubElement(location_sDc, "country", key="FR")
            settlement_sDc = ET.SubElement(location_sDc, "settlement", type="city") # que mettre ? est-ce "Paris" pour tout ?
            settlement_sDc.text = " "
            # date____________(voir partie date-pub plus bas: la date est récupérée dans chaque fichier et est ajouté ici)
            
            distributor_sDc = ET.SubElement(publicationStmt_sDc, "distributor", facs="https://gallica.bnf.fr/ark:/12148/bpt6k477552f/f1")
            distributor_sDc.text = "Source gallica.bnf.fr / Bibliothèque nationale de France"
            availability_sDc = ET.SubElement(publicationStmt_sDc, "availability")
            licence_sDc = ET.SubElement(availability_sDc, "licence", {"target":"https://gallica.bnf.fr/edit/und/conditions-dutilisation-des-contenus-de-gallica", "{http://www.w3.org/XML/1998/namespace}lang": "fr"})
            licence_sDc_p1 = ET.SubElement(licence_sDc, "p" )
            licence_sDc_p1.text = "Les contenus accessibles sur le site Gallica sont pour la plupart des reproductions numériques d'œuvres tombées dans le domaine public provenant des collections de la BnF."
            licence_sDc_p2 = ET.SubElement(licence_sDc, "p" )
            licence_sDc_p2.text = "Ces contenus sont considérés, en vertu du code des relations entre le public et l’administration, comme étant des informations publiques et leur réutilisation s'inscrit dans le cadre des dispositions prévues aux articles L. 321-1 à L. 327-1 de ce code."

            seriesStmt = ET.SubElement(biblFull, "seriesStmt")
            title_series = ET.SubElement(seriesStmt, "title" )
            title_series.text = "Journal Officiel de la République française"
            biblScope1 = ET.SubElement(seriesStmt, "biblScope")
            biblScope1.text = "Débats parlementaires"
            biblScope2 = ET.SubElement(seriesStmt, "biblScope")
            biblScope2.text = "Chambre des députés"
            idno = ET.SubElement(seriesStmt, "idno" , type="ISSN")
            idno.text = "1270-5942"
            
            
    
    # EncodingDesc _________________________________________________________________
            encodingDesc = ET.SubElement(teiHeader , "encodingDesc")
            
        # profileDesc ______________________________________________________________________
            profileDesc = ET.SubElement(teiHeader , "profileDesc")
            langUsage = ET.SubElement(profileDesc, "langUsage")
            language = ET.SubElement(langUsage, "language", ident="fr")
            language.text = "Français"
            settingDesc = ET.SubElement(profileDesc, "settingDesc")
            setting_desc = ET.SubElement(settingDesc, "setting")
            name_desc1 = ET.SubElement(setting_desc, "name", type="")    
            name_desc2 = ET.SubElement(setting_desc, "name", type="")
            name_desc3 = ET.SubElement(setting_desc, "name", type="")
           # date_desc = voir partie date plus bas
            
                       
 # Ajout de la description du projet--------------------------------------------------------
            projetDesc = ET.SubElement(encodingDesc, "projetDesc")
            p1 = ET.SubElement(projetDesc, "p")
            p1.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr" # A cause des crochets et trop de "", écrire l'attibut de cette maniène pose moins de problème.
            ref1 = ET.SubElement(p1, "ref", target = "https://www.bnf.fr/fr/les-projets-de-recherche#bnf-agoda")
            ref1.text = "AGODA "           
            ref1.tail = '''est un projet qui a pour objectif de rendre disponible au format XML-TEI les textes de débats parlementaires à la Chambre des députés au cours de la Troisième République, suivant l\''''
            ref2 = ET.SubElement(p1, "ref", target ="https://github.com/mpuren/agoda/blob/ODD/documentation/agoda_odd.xml")
            ref2.text = "ODD "
            ref2.tail = "défini pour le projet à partir des "
            ref3 = ET.SubElement(p1, "ref", target = "https://github.com/clarin-eric/parla-clarin")
            ref3.text =  "recommandations produites par Parla-CLARIN. "
            ref3.tail = ''' Dans une optique de preuve de concept, la phase 1 du projet AGODA se concentre plus particulièrement
            sur la 5ème législature (1889-1893). Les textes encodés sont d'abord extraits des documents numérisés disponibles sur '''
            ref4 = ET.SubElement(p1, "ref", target = "https://gallica.bnf.fr/ark:/12148/cb328020951/date.item")
            ref4.text = "Gallica,"
            ref4.tail = ''' la bibliothèque numérique de la Biliothèque nationale de France, puis ils sont convertis 
            en XML-TEI au moyen de scripts Python.'''
            
                            # Version anglaise ______________________________________________________________________________
                
            p2 = ET.SubElement(projetDesc, "p")
            p2.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "en"
            ref1_2 = ET.SubElement(p2, "ref", target = "https://www.bnf.fr/fr/les-projets-de-recherche#bnf-agoda")
            ref1_2.text = "AGODA "
            ref1_2.tail = '''is a project that aims to make available in XML-TEI format the texts of parliamentary debates in the
            Chamber of Deputies during the Third Republic, following the'''
            ref2_2 = ET.SubElement(p2, "ref", target ="https://github.com/mpuren/agoda/blob/ODD/documentation/agoda_odd.xml")
            ref2_2.text = "ODD "
            ref2_2.tail = " defined for the project from the "
            ref3_2 = ET.SubElement(p2, "ref", target = "https://github.com/clarin-eric/parla-clarin")
            ref3_2.text = " Parla-CLARIN recommendations "
            ref3_2.tail = ''' From a proof-of-concept perspective, phase 1 of the AGODA project focuses more specifically on the
            5th legislature (1889-1893). The encoded texts are first extracted from the digitised documents available on '''
            ref4_2 = ET.SubElement(p2, "ref", target = "https://gallica.bnf.fr/ark:/12148/cb328020951/date.item")
            ref4_2.text = "Gallica,"
            ref4_2.tail = '''  the digital library of the Biliothèque nationale de France, then they are converted into
            XML-TEI using Python scripts.'''    
                      
         # fin de la description du projet--------------------------------------------------------    
 
    # Pour tous fichiers en cours de lecture, pour chaque box du fichier, 
            for i in range(len(data)):
                
                       
            # Ensuite, si "comment" est dans la box du fichier alors:
                if "comment" in data[i]:

            # Grandes divisions des débats ___________________________________________________________________________

                    if data[i]['comment'] == 'page-number':
                        # Ajouter un élément 'page-number' avec le contenu de la clé 'text_ocr'
                        page_number = ET.SubElement(body,"pb")
                        page_number.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'page-number-ref':
                        # Ajouter un élément 'page-number-ref' avec le contenu de la clé 'text_ocr'
                        incident = ET.SubElement(body, "incident")
                        desc = ET.SubElement(incident, "desc")
                        page_number_ref = ET.SubElement(desc, "ref", attrib={"target": "#p"} )
                        page_number_ref.text = data[i]['text_ocr'] 

                    elif data[i]['comment'] == 'part head':
                        # Ajouter un élément 'part head' avec le contenu de la clé 'text_ocr'
                        div = ET.SubElement(body, "div")
                        part_head = ET.SubElement(div, "head", attrib={"type": "part"} )
                        part_head.text = data[i]['text_ocr']  

                    elif data[i]['comment'] == 'opening seg':
                        # Ajouter un élément 'opening seg' avec le contenu de la clé 'text_ocr'
                        note = ET.SubElement(body, "note", attrib={"type": "opening"})
                        opening_seg = ET.SubElement(note, "seg" )
                        opening_seg.text = data[i]['text_ocr']  

                    elif data[i]['comment'] == 'closing seg':
                        # Ajouter un élément 'closing seg' avec le contenu de la clé 'text_ocr'
                        note = ET.SubElement(body, "note")
                        closing_seg = ET.SubElement(note, "seg", attrib={"type": "closing"} )
                        closing_seg.text = data[i]['text_ocr'] 

                    elif data[i]['comment'] == 'signed seg back':
                        # Ajouter un élément 'signed seg back' avec le contenu de la clé 'text_ocr'
                        signed = ET.SubElement(body, "signed")
                        signed_seg_back = ET.SubElement(signed, "seg")
                        signed_seg_back.text = data[i]['text_ocr']  

                    elif data[i]['comment'] == 'signed seg div-end':
                        # Ajouter un élément 'signed seg div-end' avec le contenu de la clé 'text_ocr'
                        signed = ET.SubElement(body, "signed")
                        signed_seg_div_end = ET.SubElement(signed, "seg")
                        signed_seg_div_end.text = data[i]['text_ocr']  

                    elif data[i]['comment'] == 'text':
                        # Ajouter un élément 'text' avec le contenu de la clé 'text_ocr'
                        text = ET.SubElement(body)
                        text.text = data[i]['text_ocr']  

                    elif data[i]['comment'] == 'text-back':
                        # Ajouter un élément 'text-back' avec le contenu de la clé 'text_ocr'
                        #div = ET.SubElement(body, "div")
                        back = ET.SubElement(div, "back")
                        text_back = ET.SubElement(back, "div" )
                        text_back.text = data[i]['text_ocr']  

            # Paragraphes et divisions correspondants _________________________________________________________________________   

                    elif data[i]['comment'] == 'seg':
                        # Ajouter un élément 'seg' avec le contenu de la clé 'text_ocr'
                        seg = ET.SubElement(body, "seg")
                        seg.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'seg-beginning':
                        # Ajouter un élément 'seg-beginning' avec le contenu de la clé 'text_ocr'
                        seg_beginning = ET.SubElement(seg, "seg-beginning")
                        seg_beginning.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'seg-end':
                        # Ajouter un élément 'seg-end' avec le contenu de la clé 'text_ocr'
                        seg_end = ET.SubElement(seg, "seg-end")
                        seg_end.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'comment seg':
                        # Ajouter un élément 'comment seg' avec le contenu de la clé 'text_ocr'
                        note = ET.SubElement(note, "note", attrib={"type": "comment"})
                        comment_seg = ET.SubElement(note, "seg")
                        comment_seg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'comment-beginning seg':
                        # Ajouter un élément 'comment-beginning seg' avec le contenu de la clé 'text_ocr'
                        note = ET.SubElement(note, attrib={"type": "comment"})
                        comment_beginning_seg = ET.SubElement(note, "seg")
                        comment_beginning_seg.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'comment-end seg':
                        # Ajouter un élément 'comment-end seg' avec le contenu de la clé 'text_ocr'
                        note = ET.SubElement(note, attrib={"type": "comment"})
                        comment_end_seg = ET.SubElement(note, "seg")
                        comment_end_seg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'note seg':
                        # Ajouter un élément 'note seg' avec le contenu de la clé 'text_ocr'
                        note_seg = ET.SubElement(note, "seg")
                        note_seg.text = data[i]['text_ocr']   

            # Prises de parole_______________________________________________________________________________________________________

                    elif data[i]['comment'] == 'u seg':
                        # Ajouter un élément 'u-seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        u_element = ET.SubElement(body, "u")
                        u_seg = ET.SubElement(u_element, "seg")
                        u_seg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'u-beginning seg':
                        # Ajouter un élément 'u-beginning seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        u_element = ET.SubElement(body, "u")
                        u_beginning_seg = ET.SubElement(u_element, "seg")
                        u_beginning_seg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'u-end seg':
                        # Ajouter un élément 'u-end seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        u_element = ET.SubElement(body, "u")
                        u_end_seg = ET.SubElement(u_element, "seg")
                        u_end_seg.text = data[i]['text_ocr']

            # Cas particuliers ______________________________________________________________________________________________________

                    elif data[i]['comment'] == 'quote seg':
                     # Ajouter un élément 'quote seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        seg = ET.SubElement(body, "seg")
                        quote_seg = ET.SubElement(seg, "quote")
                        quote_seg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'quote-beginning seg':
                        # Ajouter un élément 'quote-beginning seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        quote_beg_seg = ET.SubElement(body, "quote")
                        quote_beg_seg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'quote-end seg':
                        # Ajouter un élément 'quote-end seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        quote_end_seg = ET.SubElement(body, "quote")
                        quote_end_seg.text = data[i]['text_ocr']          

                    elif data[i]['comment'] == 'incident':
                        # Ajouter un élément 'incident' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        u_element = ET.SubElement(body, "u")
                        incident = ET.SubElement(u_element, "seg")
                        incident.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'incident-beginning':
                        # Ajouter un élément 'incident-beginning' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        incident_beginning = ET.SubElement(incident, "incident-beginning")            
                        incident_beginning.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'incident-end':
                        # Ajouter un élément 'incident-end' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                        incident_end = ET.SubElement(incident, "incident-end")            
                        incident_end.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'table':
                        # Ajouter un élément 'table' avec le contenu de la clé 'text_ocr'
                        table = ET.SubElement(body, "table")
                        row = ET.SubElement(table, "row")
                        cell = ET.SubElement(row, "cell")
                        cell.text = data[i]['text_ocr'] 

                    elif data[i]['comment'] == 'other-sitting':
                        # Ajouter un élément 'other-sitting' avec le contenu de la clé 'text_ocr'
                        div = ET.SubElement(body, "div")
                        div1 = ET.SubElement(div, "div", attrib={"type": "other-sitting"})
                        head = ET.SubElement(div1, "head")
                        head.text = data[i]['text_ocr']

            # Éléments spécifiques à la première page________________________________________________________________________________

                    elif data[i]['comment'] == "sitting contents":
                        # Ajouter un élément 'sitting contents' avec le contenu de la clé 'text_ocr'
                        div_sitting = ET.SubElement(body, "div", attrib={"type": "sitting"})
                        div_content = ET.SubElement(div_sitting, "div", attrib={"type": "content"})
                        if data[i]['text_ocr'] == "SOMMAIRE":
                            sc_element = ET.SubElement(div_content, "head")
                            sc_element.text = data[i]['text_ocr']
                        else :
                            sc_element.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'body page-number date-pub':
                        # Ajouter un élément 'body page-number date-pub' avec le contenu de la clé 'text_ocr'
                        bp_element = ET.SubElement(body, "pb", attrib={"n": "{}".format(page_number)})
                        #body.text = data[i]['text_ocr']
                        
                    # récupération de la date de publication ___________________________________________________
                        
                        texte_date = data[i]['text_ocr'] 
                        import re
                        partie_date = re.search(r'\d{1,2} [a-zA-Z]+ \d{4}', texte_date).group()

                        # fonction pour la reconnaissance des dates. 
                        import dateparser
                        parsed_date = dateparser.parse(partie_date, languages=['fr'])
                        # Formatter la date 
                        formatted_date = parsed_date.strftime('%Y-%m-%d')

                        # Création de l'élément date avec l'attribut when
                        date_sDc = ET.SubElement(publicationStmt_sDc, "date", {"when": formatted_date})
                        date_sDc.text = partie_date
                        date_desc = ET.SubElement(setting_desc, "date", {"when": formatted_date})
                        date_desc.text = partie_date
                        
                    # ça n'a pas été simple mais à la suite de nombreuses recherches, ça fonctionne ! 
                    # ^^^^^^^^ajout de la date de publication de manière automatique ^^^^^^^^^^^^^^______________________


                    elif data[i]['comment'] == 'item':
                        # Ajouter un élément 'item' avec le contenu de la clé 'text_ocr'
                        #list_element = ET.SubElement(div_content, "list")
                        list_item = ET.SubElement(div_content, "item")
                        list_item.text = data[i]['text_ocr']
                        

                    elif data[i]['comment'] == "meeting-session meeting-legislature useless":
                        # Ajouter un élément 'meeting-session meeting-legislature' avec le contenu de la clé 'text_ocr'
                        meet_session = data[i]['text_ocr']

                        # extrait le numéro de session
                        num_session = int(meet_session.split(' ')[-1][:-4])  # 1889
                        num_legis = int(meet_session.split(' ')[0][:-1])

                        # extrait le texte de la session
                        texte_session = "Session " + meet_session.split('Session ')[1]  # ajoute le mot "Sessio
                        # extrait la première lettre du mot suivant "Session"
                        lettre_session = texte_session.split('Session ')[1][0].upper()

                        # créer la balise
                        meeting_session = ET.SubElement(publicationStmt, "meeting", n=f"E{num_session}", ana="#parla.lower\n#parla.session") # une autre manière d'écrire les attributs
                        meeting_session.text = f"{texte_session}"

                        meet_legis = meet_session.split('—')
                        meeting_legislature = ET.SubElement(publicationStmt, "meeting", n=f"{num_legis}                                                    L", ana="#parla.lower\n#parla.legislature")
                        meeting_legislature.text = meet_legis[0]
                        
                        meeting_session_t = ET.SubElement(titleStmt, "meeting", n=f"E{num_session}", ana="#parla.lower\n#parla.session") # une autre manière d'écrire les attributs
                        meeting_session_t.text = f"{texte_session}".strip(".")
                        titleStmt.insert(4, meeting_session_t)

                        meet_legis = meet_session.split('—')
                        meeting_legislature_t = ET.SubElement(titleStmt, "meeting", n=f"{num_legis}L", ana="#parla.lower\n#parla.legislature")
                        meeting_legislature_t.text = meet_legis[0].split(".")[0].replace("°", "e")
                        titleStmt.insert(5, meeting_legislature_t)

                    elif data[i]['comment'] == "meeting-sitting useless" or data[i]['comment'] == "meeting-sitting":
                        # Ajouter un élément 'meeting-sitting' avec le contenu de la clé 'text_ocr'
                        meet_sit = data[i]['text_ocr'].split('—')
                        num_seance = meet_sit[1][0:3]
                        meeting_sitting = ET.SubElement(publicationStmt, "meeting", n=f"{num_seance}", ana="#parla.lower\n#parla.sitting")
                        meeting_sitting.text = data[i]['text_ocr']
                        meeting_sitting_t = ET.SubElement(titleStmt, "meeting", n=f"{num_seance}", ana="#parla.lower\n#parla.sitting")
                        meeting_sitting_t.text = data[i]['text_ocr'].split("—")[1].strip(".").replace("°", "e").lower()
                        titleStmt.insert(6, meeting_sitting_t)

                    elif data[i]['comment'] == 'agenda':
                        # Ajouter un élément 'agenda' avec le contenu de la clé 'text_ocr'
                        div_agenda = ET.SubElement(body,"div")
                        agenda_head = ET.SubElement(div_agenda, "head", type="agenda")
                        agenda_head.text = data[i]['text_ocr']

            # Annexes________________________________________________________________________________________________________________

                    elif data[i]['comment'] == 'part1':
                    # Ajouter un élément 'part1' avec le contenu de la clé 'text_ocr'
                        div_annexe = ET.SubElement(body, "div", attrib={"type": "part"})
                    #div_annexe.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'part1 u-beginning seg':
                        # Ajouter un élément 'part1 u-beginning seg' avec le contenu de la clé 'text_ocr'
                        part_u_beg = ET.SubElement(div_annexe, "u")
                        seg_part1 = ET.SubElement(u, "seg")
                        seg_part1.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'erratum':
                        # Ajouter un élément 'erratum' avec le contenu de la clé 'text_ocr'
                        div_erratum = ET.SubElement(body, "div", attrib={"type": "erratum"})
                        head_annexe = ET.SubElement(div_erratum, "head")
                        label_annexe = ET.SubElement(head_annexe, "label")
                        label_annexe.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'note-head':
                        # Ajouter un élément 'note-head' avec le contenu de la clé 'text_ocr'
                        note_head_annexe = ET.SubElement(head_annexe,"note")
                        note_head_annexe.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'lists':
                        # Ajouter un élément 'lists' avec le contenu de la clé 'text_ocr'
                        div_lists = ET.SubElement(body, "div", attrib={"type": "lists"})
                        head_lists = ET.SubElement(div_lists, "head")
                        label_lists = ET.SubElement(head_lists, "label")
                        label_lists.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'offices': 
                        # Ajouter un élément 'offices' avec le contenu de la clé 'text_ocr'
                        div_offices = ET.SubElement(body, "div", attrib={"type": "offices"})
                        head_offices = ET.SubElement(div_erratum, "head")
                        head_offices.text = data[i]['text_ocr']     



                    elif data[i]['comment'] == 'appendices': 
                        # Ajouter un élément 'appendices' avec le contenu de la clé 'text_ocr'
                        div_appendices = ET.SubElement(body, "div", attrib={"type": "appendices"})
                        head_appendices = ET.SubElement(div_appendices, "head")
                        head_offices_appendices.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'voting1' or data[i]['comment'] == 'voting1': 
                        # Ajouter un élément 'voting1' avec le contenu de la clé 'text_ocr'
                        div_voting1 = ET.SubElement(body, "div", attrib={"type": "voting"})
                        head_voting1 = ET.SubElement(div_voting1, "head")
                        label_voting1 = ET.SubElement(head_voting1, "label")
                        label_voting1.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'result':
                        # Ajouter un élément 'result' avec le contenu de la clé 'text_ocr'
                        note_result = ET.SubElement(div_voting1, "note", attrib={"type": "result"})
                        seg_note_annexe = ET.SubElement(note_result, "seg")
                        seg_note_annexe.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'voterslist-beginning': 
                        # Ajouter un élément 'voterslist-beginning' avec le contenu de la clé 'text_ocr'
                        note_voterlist = ET.SubElement(div_voting1, "note", attrib={"type": "voterlist"})
                        voterlist = ET.SubElement(note_voterlist, "desc")
                        voterlist.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'desc':  
                        # Ajouter un élément 'desc' avec le contenu de la clé 'text_ocr'
                        voterlist_desc = ET.SubElement(note_voterlist, "desc")
                        voterlist_desc.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'seg note-beginning': 
                        # Ajouter un élément 'seg note-beginning' avec le contenu de la clé 'text_ocr'
                        note_note_beg = ET.SubElement(div_voting1, "note", attrib={"type": "numbersannounced"})
                        seg_note_beg = ET.SubElement(note_note_beg, "seg")
                        seg_note_beg.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'seg note-end' or data[i]['comment'] == 'seg note-end div-end': 
                        # Ajouter un élément 'seg note-end' avec le contenu de la clé 'text_ocr'
                        seg_note_end = ET.SubElement(div_voting1, "seg")
                        seg_note_end.text = data[i]['text_ocr']


                    #elif data[i]['comment'] == 'seg note-end': 


                    elif data[i]['comment'] == 'rectification':
                        # Ajouter un élément 'rectification' avec le contenu de la clé 'text_ocr'
                        div_rectification = ET.SubElement(body, "div", attrib={"type": "rectification"})
                        head_rectification = ET.SubElement(div_rectification, "head")
                        head_rectification.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'seg comment-beginning': 
                        # Ajouter un élément 'seg comment-beginning' avec le contenu de la clé 'text_ocr'
                        note_com_beg = ET.SubElement(body, "note", attrib={"type": "comment"})
                        seg_com_beg = ET.SubElement(note_com_beg, "seg")
                        seg_com_beg.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'seg note-end':  
                        # Ajouter un élément 'seg note-end' avec le contenu de la clé 'text_ocr'
                        seg_com_end = ET.SubElement(note_com_beg, "seg")
                        seg_com_end.text = data[i]['text_ocr']

                    elif data[i]['comment'] == 'petition':
                        # Ajouter un élément 'petition' avec le contenu de la clé 'text_ocr'
                        div_petition = ET.SubElement(body, "div", attrib={"type": "petition"})
                        head_petition = ET.SubElement(div_petition, "head")
                        label_petition = ET.SubElement(head_petition, "label")
                        label_petition.text = data[i]['text_ocr']


                    elif data[i]['comment'] == 'note-head':   
                        # Ajouter un élément 'note-head' avec le contenu de la clé 'text_ocr'
                        note_petition = ET.SubElement(head_petition, "note")
                        note_petition.text = data[i]['text_ocr']

            xml_tree = ET.ElementTree(root)
            xml_filename = os.path.splitext(filename)[0] + ".xml"
            with open(os.path.join(dossier_xml, xml_filename), "wb") as xml_file:
                root = xml_tree.getroot()

                # Insérer les instructions de modèle
                instruction1 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://purl.oclc.org/dsdl/schematron"'
                model_instr1 = ET.ProcessingInstruction("xml-model", instruction1)
                root.addprevious(model_instr1)

                instruction2 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"'
                model_instr2 = ET.ProcessingInstruction("xml-model", instruction2)
                root.addprevious(model_instr2)
               
                #xml_file.write(declaration.encode('utf-8'))
                xml_tree.write(xml_file, encoding="utf-8", xml_declaration=False, pretty_print=True)

In [2]:
import xml.etree.ElementTree as ET

xml_file = "C:/Users/brune/Desktop/stage_AGODA/FR_3R_5L_1889-11-26_p0175.xml"

tree = ET.parse(xml_file)
root = tree.getroot()

for child in root.iter():
    print(child.tag)


{http://www.tei-c.org/ns/1.0}TEI
{http://www.tei-c.org/ns/1.0}teiHeader
{http://www.tei-c.org/ns/1.0}fileDesc
{http://www.tei-c.org/ns/1.0}titleStmt
{http://www.tei-c.org/ns/1.0}title
{http://www.tei-c.org/ns/1.0}title
{http://www.tei-c.org/ns/1.0}title
{http://www.tei-c.org/ns/1.0}title
{http://www.tei-c.org/ns/1.0}meeting
{http://www.tei-c.org/ns/1.0}meeting
{http://www.tei-c.org/ns/1.0}meeting
{http://www.tei-c.org/ns/1.0}respStmt
{http://www.tei-c.org/ns/1.0}persName
{http://www.tei-c.org/ns/1.0}forename
{http://www.tei-c.org/ns/1.0}surname
{http://www.tei-c.org/ns/1.0}ptr
{http://www.tei-c.org/ns/1.0}resp
{http://www.tei-c.org/ns/1.0}resp
{http://www.tei-c.org/ns/1.0}respStmt
{http://www.tei-c.org/ns/1.0}persName
{http://www.tei-c.org/ns/1.0}forename
{http://www.tei-c.org/ns/1.0}surname
{http://www.tei-c.org/ns/1.0}ptr
{http://www.tei-c.org/ns/1.0}ptr
{http://www.tei-c.org/ns/1.0}resp
{http://www.tei-c.org/ns/1.0}resp
{http://www.tei-c.org/ns/1.0}respStmt
{http://www.tei-c.org/ns/

In [69]:
xml_file = "C:/Users/brune/Desktop/stage_AGODA/FR_3R_5L_1889-11-26_p0175.xml"

from lxml import etree

# Charger le document XML
tree = etree.parse(xml_file)

# Récupérer tous les noms de tag uniques
unique_tags = tree.xpath("//*[not(self::*)]/name()")


print(unique_tags)


XPathEvalError: Invalid expression

In [55]:
import xml.etree.ElementTree as ET

def count_tags(xml_file):
    # Créer l'élément "tagsDecl" et la sous-élément "namespace"
    tags_decl = ET.Element("tagsDecl")
    namespace = ET.SubElement(tags_decl, "namespace", {"name": "http://www.tei-c.org/ns/1.0"})

    # Parcourir le fichier XML et compter les occurrences de chaque tag
    tags_count = {}
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for elem in root.iter():
        gi = elem.tag.split("}")[1]
        tags_count[gi] = tags_count.get(gi, 0) + 1

    # Ajouter chaque tag avec son nombre d'occurrences dans la sous-élément "tagUsage"
    for gi, occurs in tags_count.items():
        ET.SubElement(namespace, "tagUsage", {"gi": gi, "occurs": str(occurs)})

    # Retourner l'élément "tagsDecl"
    return tags_decl


In [2]:


'''A NE PAS SUPPRIMER!'''


# Ce code fait du parsing et affiche le texte du fichier xml
from lxml import etree

# Charger le document XML
tree = etree.parse('mon_fichier.xml')

# Récupérer tous les noms de tag uniques
tags = tree.xpath('//*[not(self::*)]/name()')
unique_tags = list(set(tags))

print(unique_tags)
from lxml import etree

# Charger le document XML
tree = etree.parse('mon_fichier.xml')

# Récupérer tous les noms de tag uniques
tags = tree.xpath('//*[not(self::*)]/name()')
unique_tags = list(set(tags))

print(unique_tags)

import lxml.etree as ET

# Charger le fichier JSON

xml_file = "C:/Users/brune/Desktop/stage_AGODA/xml_data/FR_3R_5L_1889-11-26_p0175.xml"
    
# Charger le fichier XML
tree = ET.parse(xml_file)
root = tree.getroot()

# Fonction récursive pour extraire le texte de chaque élément XML
def extract_text(element):
    text = ""
    # Parcourir les sous-éléments de l'élément actuel
    for sub_element in element:
        # Si le sous-élément est un texte, ajouter son contenu à la chaîne de texte
        if sub_element.text:
            text += sub_element.text
        # Si le sous-élément a des sous-éléments, appeler la fonction récursivement sur ces sous-éléments
        if sub_element:
            text += extract_text(sub_element)
        # Si le sous-élément a un texte de fermeture, ajouter son contenu à la chaîne de texte
        if sub_element.tail:
            text += sub_element.tail
    return text

# Extraire le texte de l'élément racine et l'afficher
text = extract_text(root)
print(text)



    
      
        
        
        
        
        Session extraordinaire de 1889.
        5° législature. 
        
          
            Brunel
            TCHEKELI
            
          
          Transformation du JSON en XML-TEI et ajout automatique des balises TEI par des scripts Python
          Transformation from JSON to XML-TEI and automatic addition of TEI tags by Python scripts
        
        
          
            Marie
            PUREN
            
            
          
          TEI Header
          TEI Header
        
        
          
            Pierre
            VERNUS
            
          
        
        
          Bibliothèque nationale de France
          National Library of France
        
      
      
      
    
    
      
        AGODA est un projet qui a pour objectif de rendre disponible au format XML-TEI les textes de débats parlementaires à la Chambre des députés au cours de la Troisième République, suivant l'ODD défini pour le proje

C:\Users\brune\AppData\Local\Temp\ipykernel_4544\3309093751.py:25: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if sub_element:


import json
import pandas as pd

with open("FR_3R_5L_1889-11-26_p0175.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
for i in range(len(data)):
    if "comment" in data[i].keys():
        print(data[i]["comment"])


page-number
u seg
u seg
u-beginning seg
u-end seg
comment seg
u seg
u-beginning seg
u-end seg incident
u seg
u seg incident
u seg
u-beginning seg
u-end seg quote
u seg
u-beginning seg incident
seg-beginning
seg-end
seg
seg incident
seg
seg
seg
seg incident u-end
u seg
u seg
u seg incident
u-beginning seg-beginning
seg-end incident
seg
seg incident
seg
seg
seg
seg quote incident
seg
seg-beginning


In [4]:
data[55]["text_ocr"]

''

In [5]:
import os
import xml.etree.ElementTree as ET

# fonction d'extraction de texte
def extract_text(element):
    text = ""
    # Parcourir les sous-éléments de l'élément actuel
    for sub_element in element:
        # Si le sous-élément est un texte, ajouter son contenu à la chaîne de texte
        if sub_element.text:
            text += sub_element.text
        # Si le sous-élément a des sous-éléments, appeler la fonction récursivement sur ces sous-éléments
        if sub_element:
            text += extract_text(sub_element)
        # Si le sous-élément a un texte de fermeture, ajouter son contenu à la chaîne de texte
        if sub_element.tail:
            text += sub_element.tail
    return text

folder_path = r"C:/Users/brune/Desktop/stage_AGODA/xml_data"

# parcourir tous les fichiers dans le dossier xml_data
for filename in os.listdir(folder_path):
    if filename.endswith('.xml'):
        # charger le fichier XML
        xml_file = os.path.join(folder_path, filename)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # appliquer la fonction extract_text() sur chaque élément dans le fichier
        for element in root.iter():
            text = extract_text(element)
            # faire quelque chose avec le texte extrait, par exemple, l'enregistrer dans une base de données ou un fichier texte

            if text:
                print(text)


    
      
        
        
        
        
        Session extraordinaire de 1889.
        5° législature. 
        
          
            Brunel
            TCHEKELI
            
          
          Transformation du JSON en XML-TEI et ajout automatique des balises TEI par des scripts Python
          Transformation from JSON to XML-TEI and automatic addition of TEI tags by Python scripts
        
        
          
            Marie
            PUREN
            
            
          
          TEI Header
          TEI Header
        
        
          
            Pierre
            VERNUS
            
          
        
        
          Bibliothèque nationale de France
          National Library of France
        
      
      
      
    
    
      
        AGODA est un projet qui a pour objectif de rendre disponible au format XML-TEI les textes de débats parlementaires à la Chambre des députés au cours de la Troisième République, suivant l'ODD défini pour le proje

In [6]:
# Ce code vérifie la validité du fichier et affiche la racine si tout est ok

#import xml.etree.ElementTree as ET

# Définir le chemin du fichier XML
xml_file = "votre_fichier.xml"

# Charger le fichier XML
try:
    tree = ET.parse(xml_file)
except ET.ParseError as e:
    print(f"Erreur de syntaxe XML : {e}")
    # Sortir du script si le fichier XML contient une erreur de syntaxe
    exit()

# Récupérer la racine de l'arbre XML
root = tree.getroot()

# Afficher la racine de l'arbre XML
print(root.tag)

# Si le fichier XML est valide, le script va afficher la balise de la racine de l'arbre XML


{http://www.tei-c.org/ns/1.0}TEI


In [ ]:
import json, os, re
import lxml.etree as ET



# Chargement des fichiers JSON
with open("FR_3R_5L_1889-11-26_p0175.json", "r", encoding="utf-8") as f:
    data = json.load(f)

#Créer un élément racine
root = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
teiHeader = ET.SubElement(root, "teiHeader")
text_tei = ET.SubElement(root, "text")
body = ET.SubElement(text_tei, "body")

"""Petit scrpit qui recupère le numéro de page de chaque fichier:

nom_fichier = "FR_3R_5L_1889-11-26_p0175.json"
pag = nom_fichier.split("_p0")
page = pag[-1].split(".")
page[0]
# A automatiser plus tard
"""
nom_fichier = "FR_3R_5L_1889-11-26_p0175.json"
pag = nom_fichier.split("_p0")
page = pag[-1].split(".")
page[0]


for i in range(len(data)):
    if "comment" in data[i].keys():

# Grandes divisions des débats ___________________________________________________________________________

        if data[i]['comment'] == 'page-number':
            # Ajouter un élément 'page-number' avec le contenu de la clé 'text_ocr'
            page_number = ET.SubElement(body,"pb")
            page_number.text = data[i]['text_ocr']
                
        elif data[i]['comment'] == 'page-number-ref':
            # Ajouter un élément 'page-number-ref' avec le contenu de la clé 'text_ocr'
            incident = ET.SubElement(body, "incident")
            desc = ET.SubElement(incident, "desc")
            page_number_ref = ET.SubElement(desc, "ref", attrib={"target": "#p"} )
            page_number_ref.text = data[i]['text_ocr'] 

        elif data[i]['comment'] == 'part head':
            # Ajouter un élément 'part head' avec le contenu de la clé 'text_ocr'
            div = ET.SubElement(body, "div")
            part_head = ET.SubElement(div, "head", attrib={"type": "part"} )
            part_head.text = data[i]['text_ocr']  
            
        elif data[i]['comment'] == 'opening seg':
            # Ajouter un élément 'opening seg' avec le contenu de la clé 'text_ocr'
            note = ET.SubElement(body, "note", attrib={"type": "opening"})
            opening_seg = ET.SubElement(note, "seg" )
            opening_seg.text = data[i]['text_ocr']  
        
        elif data[i]['comment'] == 'closing seg':
            # Ajouter un élément 'closing seg' avec le contenu de la clé 'text_ocr'
            note = ET.SubElement(body, "note")
            closing_seg = ET.SubElement(note, "seg", attrib={"type": "closing"} )
            closing_seg.text = data[i]['text_ocr'] 
            
        elif data[i]['comment'] == 'signed seg back':
            # Ajouter un élément 'signed seg back' avec le contenu de la clé 'text_ocr'
            signed = ET.SubElement(body, "signed")
            signed_seg_back = ET.SubElement(signed, "seg")
            signed_seg_back.text = data[i]['text_ocr']  
            
        elif data[i]['comment'] == 'signed seg div-end':
            # Ajouter un élément 'signed seg div-end' avec le contenu de la clé 'text_ocr'
            signed = ET.SubElement(body, "signed")
            signed_seg_div_end = ET.SubElement(signed, "seg")
            signed_seg_div_end.text = data[i]['text_ocr']  
        
        elif data[i]['comment'] == 'text':
            # Ajouter un élément 'text' avec le contenu de la clé 'text_ocr'
            text = ET.SubElement(body)
            text.text = data[i]['text_ocr']  
        
        elif data[i]['comment'] == 'text-back':
            # Ajouter un élément 'text-back' avec le contenu de la clé 'text_ocr'
            #div = ET.SubElement(body, "div")
            back = ET.SubElement(div, "back")
            text_back = ET.SubElement(back, "div" )
            text_back.text = data[i]['text_ocr']  
            
# Paragraphes et divisions correspondants _________________________________________________________________________   

        elif data[i]['comment'] == 'seg':
            # Ajouter un élément 'seg' avec le contenu de la clé 'text_ocr'
            seg = ET.SubElement(body, "seg")
            seg.text = data[i]['text_ocr']


        elif data[i]['comment'] == 'seg-beginning':
            # Ajouter un élément 'seg-beginning' avec le contenu de la clé 'text_ocr'
            seg_beginning = ET.SubElement(seg, "seg-beginning")
            seg_beginning.text = data[i]['text_ocr']


        elif data[i]['comment'] == 'seg-end':
            # Ajouter un élément 'seg-end' avec le contenu de la clé 'text_ocr'
            seg_end = ET.SubElement(seg, "seg-end")
            seg_end.text = data[i]['text_ocr']


        elif data[i]['comment'] == 'comment seg':
            # Ajouter un élément 'comment seg' avec le contenu de la clé 'text_ocr'
            note = ET.SubElement(note, attrib={"type": "comment"})
            comment_seg = ET.SubElement(note, "seg")
            comment_seg.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'comment-beginning seg':
            # Ajouter un élément 'comment-beginning seg' avec le contenu de la clé 'text_ocr'
            note = ET.SubElement(note, attrib={"type": "comment"})
            comment_beginning_seg = ET.SubElement(note, "seg")
            comment_beginning_seg.text = data[i]['text_ocr']


        elif data[i]['comment'] == 'comment-end seg':
            # Ajouter un élément 'comment-end seg' avec le contenu de la clé 'text_ocr'
            note = ET.SubElement(note, attrib={"type": "comment"})
            comment_end_seg = ET.SubElement(note, "seg")
            comment_end_seg.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'note seg':
            # Ajouter un élément 'note seg' avec le contenu de la clé 'text_ocr'
            note_seg = ET.SubElement(note, "seg")
            note_seg.text = data[i]['text_ocr']   

# Prises de parole_______________________________________________________________________________________________________

        elif data[i]['comment'] == 'u seg':
            # Ajouter un élément 'u-seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            u_element = ET.SubElement(body, "u")
            u_seg = ET.SubElement(u_element, "seg")
            u_seg.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'u-beginning seg':
            # Ajouter un élément 'u-beginning seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            u_element = ET.SubElement(body, "u")
            u_beginning_seg = ET.SubElement(u_element, "seg")
            u_beginning_seg.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'u-end seg':
            # Ajouter un élément 'u-end seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            u_element = ET.SubElement(body, "u")
            u_end_seg = ET.SubElement(u_element, "seg")
            u_end_seg.text = data[i]['text_ocr']

# Cas particuliers ______________________________________________________________________________________________________

        elif data[i]['comment'] == 'quote seg':
         # Ajouter un élément 'quote seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            seg = ET.SubElement(body, "seg")
            quote_seg = ET.SubElement(seg, "quote")
            quote_seg.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'quote-beginning seg':
            # Ajouter un élément 'quote-beginning seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            quote_beg_seg = ET.SubElement(body, "quote")
            quote_beg_seg.text = data[i]['text_ocr']
            
        elif data[i]['comment'] == 'quote-end seg':
            # Ajouter un élément 'quote-end seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            quote_end_seg = ET.SubElement(body, "quote")
            quote_end_seg.text = data[i]['text_ocr']          
            
        elif data[i]['comment'] == 'incident':
            # Ajouter un élément 'incident' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            u_element = ET.SubElement(body, "u")
            incident = ET.SubElement(u_element, "seg")
            incident.text = data[i]['text_ocr']
            
            
        elif data[i]['comment'] == 'incident-beginning':
            # Ajouter un élément 'incident-beginning' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            incident_beginning = ET.SubElement(incident, "incident-beginning")            
            incident_beginning.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'incident-end':
            # Ajouter un élément 'incident-end' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
            incident_end = ET.SubElement(incident, "incident-end")            
            incident_end.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'table':
            # Ajouter un élément 'table' avec le contenu de la clé 'text_ocr'
            table = ET.SubElement(body, "table")
            row = ET.SubElement(table, "row")
            cell = ET.SubElement(row, "cell")
            cell.text = data[i]['text_ocr'] 
       
        elif data[i]['comment'] == 'other-sitting':
            # Ajouter un élément 'other-sitting' avec le contenu de la clé 'text_ocr'
            div = ET.SubElement(body, "div")
            div1 = ET.SubElement(div, "div", attrib={"type": "other-sitting"})
            head = ET.SubElement(div1, "head")
            head.text = data[i]['text_ocr']

# Éléments spécifiques à la première page________________________________________________________________________________

        elif data[i]['comment'] == "sitting contents":
            # Ajouter un élément 'sitting contents' avec le contenu de la clé 'text_ocr'
            div_sitting = ET.SubElement(body, "div", attrib={"type": "sitting"})
            div_content = ET.SubElement(div_sitting, "div", attrib={"type": "content"})
            if data[i]['text_ocr'] == "SOMMAIRE":
                sc_element = ET.SubElement(div_content, "head")
                sc_element.text = data[i]['text_ocr']
            else :
                sc_element.text = data[i]['text_ocr']

        elif data[i]['comment'] == 'body page-number date-pub':
            # Ajouter un élément 'body page-number date-pub' avec le contenu de la clé 'text_ocr'
            bp_element = ET.SubElement(body, "pb", attrib={"n": "{}".format(page[0])})
            body.text = data[i]['text_ocr']
             # la récupération de la page fonctionne. on peut faire qqch de similaire avc le 1er "pb" que nous avons récupéré
            # Si besoin, je crée une sous balise head pour contenir l'en-tête 
        # récupération de la date de publication ___________________________________________________
            texte_date = data[i]['text_ocr'] #
            import re
            partie_date = re.search(r'\d{1,2} [a-zA-Z]+ \d{4}', texte_date).group()
            
            publicationStmt = ET.SubElement(body, "publicationStmt")
            publisher = ET.SubElement(publicationStmt, "publisher")
            pubPlace= ET.SubElement(publicationStmt, "pubPlace")
            date = ET.SubElement(publicationStmt, "date")
            date.text = partie_date 
        # ^^^^^^^^ajout de la date de publication de manière automatique ^^^^^^^^^^^^^^______________________


        elif data[i]['comment'] == 'item':
            # Ajouter un élément 'item' avec le contenu de la clé 'text_ocr'
            list_element = ET.SubElement(div_content, "list")
            list_item = ET.SubElement(list_element, "item")
            list_item.text = "-" + data[i]['text_ocr']
            
            
        elif data[i]['comment'] == "meeting-session meeting-legislature useless":
            # Ajouter un élément 'meeting-session meeting-legislature' avec le contenu de la clé 'text_ocr'
            meet_session = data[i]['text_ocr']

            # extrait le numéro de session
            num_session = int(meet_session.split(' ')[-1][:-4])  # 1889
            num_legis = int(meet_session.split(' ')[0][:-1])

            # extrait le texte de la session
            texte_session = "Session " + meet_session.split('Session ')[1]  # ajoute le mot "Session"

            # extrait la première lettre du mot suivant "Session"
            lettre_session = texte_session.split('Session ')[1][0].upper()

            # créer la balise
            meeting_session = ET.SubElement(publicationStmt, "meeting", n=f"E{num_session}", ana="#parla.lower\n#parla.session") # une autre manière d'écrire les attributs
            meeting_session.text = f"{texte_session}"
            
            meet_legis = meet_session.split('—')
            meeting_legislature = ET.SubElement(publicationStmt, "meeting", n=f"{num_legis}                                                    L", ana="#parla.lower\n#parla.legislature")
            meeting_legislature.text = meet_legis[0]
            
        elif data[i]['comment'] == "meeting-sitting useless" or data[i]['comment'] == "meeting-sitting":
            # Ajouter un élément 'meeting-sitting' avec le contenu de la clé 'text_ocr'
            meet_sit = data[i]['text_ocr'].split('—')
            num_seance = meet_sit[1][0:3]
            meeting_sitting = ET.SubElement(publicationStmt, "meeting", n=f"{num_seance}", ana="#parla.lower\n#parla.sitting")
            meeting_sitting.text = data[i]['text_ocr']

       
        elif data[i]['comment'] == 'agenda':
            # Ajouter un élément 'agenda' avec le contenu de la clé 'text_ocr'
            div_agenda = ET.SubElement(body,"div")
            agenda_head = ET.SubElement(div_agenda, "head", type="agenda")
            agenda_head.text = data[i]['text_ocr']

# Annexes________________________________________________________________________________________________________________

        elif data[i]['comment'] == 'part1':
        # Ajouter un élément 'part1' avec le contenu de la clé 'text_ocr'
            div_annexe = ET.SubElement(body, "div", attrib={"type": "part"})
        #div_annexe.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'part1 u-beginning seg':
            # Ajouter un élément 'part1 u-beginning seg' avec le contenu de la clé 'text_ocr'
            part_u_beg = ET.SubElement(div_annexe, "u")
            seg_part1 = ET.SubElement(u, "seg")
            seg_part1.text = data[i]['text_ocr']
        
        elif data[i]['comment'] == 'erratum':
            # Ajouter un élément 'erratum' avec le contenu de la clé 'text_ocr'
            div_erratum = ET.SubElement(body, "div", attrib={"type": "erratum"})
            head_annexe = ET.SubElement(div_erratum, "head")
            label_annexe = ET.SubElement(head_annexe, "label")
            label_annexe.text = data[i]['text_ocr']
            
        
        elif data[i]['comment'] == 'note-head':
            # Ajouter un élément 'note-head' avec le contenu de la clé 'text_ocr'
            note_head_annexe = ET.SubElement(head_annexe,"note")
            note_head_annexe.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'lists':
            # Ajouter un élément 'lists' avec le contenu de la clé 'text_ocr'
            div_lists = ET.SubElement(body, "div", attrib={"type": "lists"})
            head_lists = ET.SubElement(div_lists, "head")
            label_lists = ET.SubElement(head_lists, "label")
            label_lists.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'offices': 
            # Ajouter un élément 'offices' avec le contenu de la clé 'text_ocr'
            div_offices = ET.SubElement(body, "div", attrib={"type": "offices"})
            head_offices = ET.SubElement(div_erratum, "head")
            head_offices.text = data[i]['text_ocr']     
            
            
            
        elif data[i]['comment'] == 'appendices': 
            # Ajouter un élément 'appendices' avec le contenu de la clé 'text_ocr'
            div_appendices = ET.SubElement(body, "div", attrib={"type": "appendices"})
            head_appendices = ET.SubElement(div_appendices, "head")
            head_offices_appendices.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'voting1' or data[i]['comment'] == 'voting1': 
            # Ajouter un élément 'voting1' avec le contenu de la clé 'text_ocr'
            div_voting1 = ET.SubElement(body, "div", attrib={"type": "voting"})
            head_voting1 = ET.SubElement(div_voting1, "head")
            label_voting1 = ET.SubElement(head_voting1, "label")
            label_voting1.text = data[i]['text_ocr']
        
              
        elif data[i]['comment'] == 'result':
            # Ajouter un élément 'result' avec le contenu de la clé 'text_ocr'
            note_result = ET.SubElement(div_voting1, "note", attrib={"type": "result"})
            seg_note_annexe = ET.SubElement(note_result, "seg")
            seg_note_annexe.text = data[i]['text_ocr']
            
        elif data[i]['comment'] == 'voterslist-beginning': 
            # Ajouter un élément 'voterslist-beginning' avec le contenu de la clé 'text_ocr'
            note_voterlist = ET.SubElement(div_voting1, "note", attrib={"type": "voterlist"})
            voterlist = ET.SubElement(note_voterlist, "desc")
            voterlist.text = data[i]['text_ocr']
                
        
        elif data[i]['comment'] == 'desc':  
            # Ajouter un élément 'desc' avec le contenu de la clé 'text_ocr'
            voterlist_desc = ET.SubElement(note_voterlist, "desc")
            voterlist_desc.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'seg note-beginning': 
            # Ajouter un élément 'seg note-beginning' avec le contenu de la clé 'text_ocr'
            note_note_beg = ET.SubElement(div_voting1, "note", attrib={"type": "numbersannounced"})
            seg_note_beg = ET.SubElement(note_note_beg, "seg")
            seg_note_beg.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'seg note-end' or data[i]['comment'] == 'seg note-end div-end': 
            # Ajouter un élément 'seg note-end' avec le contenu de la clé 'text_ocr'
            seg_note_end = ET.SubElement(div_voting1, "seg")
            seg_note_end.text = data[i]['text_ocr']
            
        
        #elif data[i]['comment'] == 'seg note-end': 
            
                
        elif data[i]['comment'] == 'rectification':
            # Ajouter un élément 'rectification' avec le contenu de la clé 'text_ocr'
            div_rectification = ET.SubElement(body, "div", attrib={"type": "rectification"})
            head_rectification = ET.SubElement(div_rectification, "head")
            head_rectification.text = data[i]['text_ocr']
        
        
        elif data[i]['comment'] == 'seg comment-beginning': 
            # Ajouter un élément 'seg comment-beginning' avec le contenu de la clé 'text_ocr'
            note_com_beg = ET.SubElement(body, "note", attrib={"type": "comment"})
            seg_com_beg = ET.SubElement(note_com_beg, "seg")
            seg_com_beg.text = data[i]['text_ocr']
        
        elif data[i]['comment'] == 'seg note-end':  
            # Ajouter un élément 'seg note-end' avec le contenu de la clé 'text_ocr'
            seg_com_end = ET.SubElement(note_com_beg, "seg")
            seg_com_end.text = data[i]['text_ocr']
        
        elif data[i]['comment'] == 'petition':
            # Ajouter un élément 'petition' avec le contenu de la clé 'text_ocr'
            div_petition = ET.SubElement(body, "div", attrib={"type": "petition"})
            head_petition = ET.SubElement(div_petition, "head")
            label_petition = ET.SubElement(head_petition, "label")
            label_petition.text = data[i]['text_ocr']
    
        
        elif data[i]['comment'] == 'note-head':   
            # Ajouter un élément 'note-head' avec le contenu de la clé 'text_ocr'
            note_petition = ET.SubElement(head_petition, "note")
            note_petition.text = data[i]['text_ocr']
        
     
# Créer un nouvel arbre XML
xml_tree = ET.ElementTree(root)

# Enregistrer le fichier XML
xml_tree.write("votre_fichier.xml", encoding="utf-8", pretty_print=True)


In [ ]:
import json
import os
import lxml.etree as ET

# Chemin du dossier contenant les fichiers à parcourir
dossier_json = "C:/Users/brune/Desktop/stage_AGODA/json_data"
dossier_xml = "C:/Users/brune/Desktop/stage_AGODA/xml_data"

# Itérer sur tous les fichiers dans le dossier
for filename in os.listdir(dossier_json):
    # Vérifier si le fichier est un fichier JSON
    if filename.endswith(".json"):
        # Ouvrir le fichier JSON
        with open(os.path.join(dossier_json, filename), "r") as json_file:
            # Lire le contenu du fichier JSON
            data = json.load(json_file)

            #Créer un élément racine
            root = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
            teiHeader = ET.SubElement(root, "teiHeader")
            text_tei = ET.SubElement(root, "text")
            body = ET.SubElement(text_tei, "body")

            for item in data:
                if "comment" in item:
                    # Grandes divisions des débats ___________________________________________________________________________

                    if item['comment'] == 'page-number':
                        # Ajouter un élément 'page-number' avec le contenu de la clé 'text_ocr'
                        page_number = ET.SubElement(body,"pb")
                        page_number.text = item['text_ocr']

                    elif item['comment'] == 'page-number-ref':
                        # Ajouter un élément 'page-number-ref' avec le contenu de la clé 'text_ocr'
                        incident = ET.SubElement(body, "incident")
                        desc = ET.SubElement(incident, "desc")
                        page_number_ref = ET.SubElement(desc, "ref", attrib={"target": "#p"} )
                        page_number_ref.text = item['text_ocr'] 

            # Enregistrer le fichier XML
            xml_filename = os.path.splitext(filename)[0] + ".xml"
            xml_tree = ET.ElementTree(root)
            with open(os.path.join(dossier_xml, xml_filename), "wb") as xml_file:
                xml_tree.write(xml_file)


In [ ]:
filename  = "FR_3R_5L_1889-11-26_p0175.json"
page_number = filename.split(".")[0]
page_number